<a href="https://colab.research.google.com/github/bmercer486/tam470-tutorials/blob/main/point_jacobi_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Point Jacobi Solver for 2D steady heat equation

## The example

This example presents code to numerically solve for the distribution of $T(x,y)$ in the plate with the BCs shown below. You can assume Dirichlet boundary conditions take precedence over the Neumann boundary conditions at the corners of the domain.

<p align='center'>
	<img src="https://github.com/bmercer486/tam470-tutorials/blob/main/plate2d.png?raw=1" alt="plate2d" width="300"/>
</p>

The governing PDE is

<br>
\begin{equation}
\frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y ^2} = 0
\end{equation}
<br>

The discretization assumes using $N+1$ points in each of the $x$ and $y$ directions, resulting in uniform grid spacing $h$. The discretized equation for interior nodes is:

<br>
\begin{equation}
-4T_{i,j} + T_{i+1,j} + T_{i-1,j} + T_{i,j+1} + T_{i,j-1} = 0, \quad i = 1, 2, ..., N-1, \quad j = 1, 2, ..., N-1
\end{equation}
<br>

On the right edge, where $i=N$, the insulated boundary condition imposed with a first order difference scheme gives

<br>
\begin{equation}
T_{N,j} = T_{N-1,j}, \quad j = 1, 2, ..., N-1
\end{equation}
<br>

Substituting this into the discretized Laplacian equation above implies the following equations are solved for nodes adjacent to the right boundary:

<br>
$$
-3T_{i,j} + T_{i-1,j} + T_{i,j+1} + T_{i,j-1} = 0
$$
<br>

## Point Jacobi implementation

To initialize the Point Jacobi scheme, pick an initial guess to kick off the iterations, $k=0$. All zeros is a common choice, with the exception of including known Dirichlet boundary values in the guess:

<br>
\begin{equation}
T^{k=0}(x, y) = \begin{cases}
			x(L-x)       & \text{if $y = 0$},\\
			0            & \text{otherwise}
		\end{cases}
\end{equation}
<br>

If each equation in the system of equations is written in the format

<br>
$$
A_{ii}x_i +\sum_{j\ne i} A_{ij}x_j = b_i
$$
<br>

Then the Point Jacobi update rule is

<br>
$$
x_i^{(k+1)} = \frac{1}{A_{ii}}\left(b_i-\sum_{j\ne i} A_{ij}x_j^{(k)}\right)
$$
<br>

### Point Jacobi Update Rules

<b>For interior nodes adjacent to the Neumann boundary:</b> $i = N-1, \quad j = 1, 2, ..., N-1$

The equations for these nodes are given by

<br>
$$
-3T_{i,j} + T_{i-1,j} + T_{i,j+1} + T_{i,j-1} = 0
$$
<br>

So the Point Jacobi udpate rule for these nodes is

<br>
$$ T^{k+1}_{i,j} = \frac{1}{3}\left(T^k_{i-1,j} + T^k_{i,j+1} + T^k_{i,j-1}\right)$$
<br>

<b>For all other interior nodes:</b> $i = 1,2,...,N-2, \quad j = 1, 2, ..., N-1$

The equations for these nodes are given by

<br>
$$
-4T_{i,j} + T_{i+1,j} + T_{i-1,j} + T_{i,j+1} + T_{i,j-1} = 0
$$
<br>

So the update rule for these nodes is

<br>
$$
T^{k+1}_{i,j} = \frac{1}{4}\left(T^k_{i+1,j} + T^k_{i-1,j} + T^k_{i,j+1} + T^k_{i,j-1}\right)
$$
<br>

<b>Boundary node solution values</b>

Dirichlet conditions are imposed at the outset.

After convergence is achieved for interior nodes, the Neumann BC is applied to nodes along the right boundary to set their values:

<br>
\begin{equation}
T_{N,j} = T_{N-1,j}, \quad j = 1, 2, ..., N-1
\end{equation}
<br>

### Code implementation
The code below implements this scheme.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def PJitersolve(L, N, tol, maxiter, print_iters=False):

    # Discretization
    x = np.linspace(0, L, N+1)
    y = np.linspace(0, L, N+1)

    # Initialize
    T = np.zeros((N+1, N+1))
    T[:,0] = x*(L - x) # Incorporate known Dirichlet BCs into the initial guess

    # Track convergence
    dvals = np.zeros(maxiter)

    # Point Jacobi
    converged = False
    iter = 0
    if(print_iters):
        print('Iteration k:   |T(k+1) - T(k)|')
    while(not converged):

        # The 'old' guess (iteration k) is saved as the result from the previous guess
        Told = np.copy(T)

        # Iteration counter
        iter += 1

        # Loop over all interior nodes
        for i in range(1, N):
            for j in range(1, N):

                # Interior nodes affected by the Neumann condition: i=N-1
                if(i==N-1):
                    T[i,j] = (1/3)*(Told[i-1,j] + Told[i,j+1] + Told[i,j-1])

                # All other interior nodes
                else:
                    T[i,j] = (1/4)*(Told[i+1,j] + Told[i-1,j] + Told[i,j+1] + Told[i,j-1])

        # Check if converged (apply norm to interior nodes only)
        d = np.linalg.norm(T[1:-1, 1:-1] - Told[1:-1, 1:-1])
        dvals[iter-1] = d

        # Print convergence behavior (helpful for debugging)
        if(print_iters and iter % 10 == 0):
            print(f'{iter:.0f} {d:25.8f}')

        # Accept or reject current solution
        if(d < tol):
            converged = True
            print(f'Converged in {iter} iterations')
            print(f'|T(final)-T(previous)|= {d:.12f}')

            # Impose the Neumann condition on the right edge where i=N
            for j in range(1, N):
                T[N, j] = T[N-1, j]

        elif(iter==maxiter):
            print('no convergence')
            break

    return x, y, T, iter, dvals

# Domain and discretization
L = 10
N = 20

# Convergence criteria
tol = 1e-6
maxiter = 10000

# Point Jacobi
x, y, T, iter, dvals = PJitersolve(L, N, tol, maxiter, print_iters=False)

# Plot solution
X, Y = np.meshgrid(x, y)
plt.figure()
contour_regions = np.linspace(0, np.max(T), 11)
plt.contourf(X, Y, T.T, levels=contour_regions, cmap='rainbow')
plt.xlabel('x')
plt.ylabel('y')
cbar = plt.colorbar()
cbar.set_label('T', rotation=90)
plt.grid()

# Plot convergence
plt.figure()
plt.loglog(range(1, iter+1), dvals[0:iter])
plt.title(r'$d = |T^{k+1} - T^k|$ vs iterations')
plt.grid()
plt.xlabel('Number of iterations')
plt.ylabel('d')

## Speeding up the iterative loop

Just-In-Time (JIT) compilation is a technique that compiles code into machine code while a program is running, rather than ahead of time. It works by identifying frequently used sections of code and converting them to optimized machine code at runtime to improve performance.

We can do this with Python (interpreted language) with the <code>numba</code> library. We simply import the library and add the dectorator <code>@jit</code> before a function definition to JIT compile it. Just be sure to only use <code>numpy</code> libraries within the function (<code>numba</code> does not necessarily support other libraries).

Below is a new function <code>PJitersolve_Fast</code> which uses JIT. It is identical to the previous function except f-strings for printing are removed since these are not supported by <code>numba</code>.

The script following the function definition compares the previous function to the JIT version in terms of execution time. You should notice significant speedup especially for $N>20$ panels in each direction. The speedup improves with increasing $N$.

In [ ]:
import time
from numba import jit

@jit
def PJitersolve_Fast(L, N, tol, maxiter):

    # Discretization
    x = np.linspace(0, L, N+1)
    y = np.linspace(0, L, N+1)

    # Initialize
    T = np.zeros((N+1, N+1))
    T[:,0] = x*(L - x) # Incorporate known Dirichlet BCs into the initial guess

    # Track convergence
    dvals = np.zeros(maxiter)

    # Point Jacobi
    converged = False
    iter = 0
    while(not converged):

        # The 'old' guess (iteration k) is saved as the result from the previous guess
        Told = np.copy(T)

        # Iteration counter
        iter += 1

        # Loop over all interior nodes
        for i in range(1, N):
            for j in range(1, N):

                # Interior nodes affected by the Neumann condition: i=N-1
                if(i==N-1):
                    T[i,j] = (1/3)*(Told[i-1,j] + Told[i,j+1] + Told[i,j-1])

                # All other interior nodes
                else:
                    T[i,j] = (1/4)*(Told[i+1,j] + Told[i-1,j] + Told[i,j+1] + Told[i,j-1])

        # Check if converged (apply norm to interior nodes only)
        d = np.linalg.norm(T[1:-1, 1:-1] - Told[1:-1, 1:-1])
        dvals[iter-1] = d

        # Accept or reject current solution
        if(d < tol):
            converged = True
            # print('Converged in', iter, 'iterations')
            # print('|T(final)-T(previous)|=', d)

            # Impose the Neumann condition on the right edge where i=N
            for j in range(1, N):
                T[N, j] = T[N-1, j]

        elif(iter==maxiter):
            # print('no convergence')
            break

    return x, y, T, iter, dvals

# Domain and discretization
L = 10
N = 40

# Convergence criteria
tol = 1e-6
maxiter = 10000

print(f"Using N = {N} and tol={tol}")

# Point Jacobi
start = time.perf_counter() # Start timer
x, y, T, iter, dvals = PJitersolve(L, N, tol, maxiter, print_iters=False)
end = time.perf_counter() # End timer
print(f"Elapsed wall clock time (without JIT): {end - start:.4f} seconds")

# Point Jacobi with just-in-time compilation
start = time.perf_counter() # Start timer
x, y, T, iter, dvals = PJitersolve_Fast(L, N, tol, maxiter)
end = time.perf_counter() # End timer
print(f"Elapsed wall clock time (*with* JIT) : {end - start:.4f} seconds")
